> Как подключиться к базе данных
> Как подключиться к PostgreSQL через psycopg2

1. Импортируем библиотеку:

```bash
pip install psycopg2-binary если библиотека не установлена
import psycopg2
```
2. Далее используем данные для входа
```python
database="startml",
user="robot-startml-ro",
password="pheiph0hahj1Vaif",
host="postgres.lab.karpov.courses",
port=6432
```
 В конце работы необходимо закрывать курсор и соединение с БД:
```python
cursor.close() 
connection.close()
```

> Как подключиться к PostgreSQL через pandas

Для загрузки таблицы используется функция pd.read_sql.
```python
import pandas as pd

df = pd.read_sql(
    """SELECT * FROM "feed_action" LIMIT 10 """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df.head()
```

Обратите внимание, что сам запрос выделяется тройными кавычками (""") — этот знак в Python для многострочного ввода. Таким же образом выделяются многострочные комментарии в коде.



В следующих заданиях мы будем работать с базой данных, в которой имеется три таблицы:

`1. Таблица с данными о пользователях (user):`

id - уникальный идентификатор пользователя (primary key)

gender - пол

age - возраст

country - страна

city - город

exp_group - экспериментальная группа

os - операционная система

source - источник трафика

`2. Таблица с данными о постах (post):`

id - уникальный идентификатор поста (primary key)

text - текст поста

topic - тема поста

`3. Таблица с данными о действиях пользователей (feed_action):`

user_id (——>) user (id)- идентификатор пользователя     

post_id (——>) post (id)- идентификатор поста     

action - совершенное в сети действие     

time- время действия

In [8]:
# устанавливаем библиотеки
# !pip install sqlalchemy
# !pip install psycopg2-binary

import pandas as pd

df = pd.read_sql(
    """SELECT * FROM "user" LIMIT 10 """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)
# проверяем связь, выводим ответ в df
df.head()

,id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


## 1
> SELECT (1/2)

Сделайте запрос на все колонки из таблицы user. Для этого может понадобиться взять user в кавычки, вот так: "user" - чтобы PostgreSQL не спутал ее со своей встроенной таблицей user, где он ведет учет всех пользователей.

Напишите через пробел, с какого id начинаются записи и какой город у самого первого ID (на английском)

In [9]:
import pandas as pd

df1 = pd.read_sql(
    """SELECT id, city FROM "user" LIMIT 1 """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df1.head()

,id,city
0,200,Degtyarsk


## 2

> SELECT (2/2)

Отберите уникальные значения тем (topic) в таблице постов (post).

Напишите через пробел количество записей и самое длинное название топика.

In [39]:

df2 = pd.read_sql(
    """
SELECT 
    DISTINCT(topic)
    --COUNT(DISTINCT(topic))
FROM post
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df2.head(10)

,topic
0,tech
1,covid
2,movie
3,entertainment
4,politics
5,sport
6,business


## 3
> Фильтры (1/2)

- Отберите пользователей старше 30 с устройством на iOS. 
- Каков ID и город записи с минимальным ID (он идет первым в таблице)? 
- Запишите значения через пробел.

In [17]:
df3 = pd.read_sql(
    """
SELECT 
    id, city
FROM "user"
    WHERE age > 30
    AND os = 'iOS'
LIMIT 1
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df3.head(1)

,id,city
0,212,Podolsk


## 4

> Фильтры (2/2)

- Отберите пользователей, которые не из России 
- и при этом либо их экспериментальная группа не 0 и не 3 
- или их город – Минск.

- Напишите через пробел первые 4 ID в такой выдаче.

In [29]:
df4 = pd.read_sql(
    """
SELECT 
    id
FROM "user"
WHERE country != 'Russia'
    AND exp_group not in ('0', '3')
    OR city = 'Minsk'
LIMIT 4
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

res4 = df4.id.to_list()
str(res4).replace(',',' ')

'[211  213  216  218]'

## 5

> Группировка (1/6)

- Подсчитайте средний возраст пользователей в разрезе страны. 
- В ответ напишите средний возраст пользователей из Кипра. 
- Ответ округлите до двух знаков после точки.

In [32]:
df5 = pd.read_sql(
    """
SELECT 
    ROUND(AVG(age),2) as "mean_age", 
    country
FROM "user"
WHERE country = 'Cyprus'
GROUP BY country
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df5.head()

,mean_age,country
0,28.07,Cyprus


## 6

> Группировка (2/6)

- Сгруппируйте пользователей сначала по экспериментальной группе, 
- а затем по операционной системе. 
- В каждой группе подсчитайте количество пользователей, максимальный и минимальный возраст.

- Сохраните результаты в файл CSV колонками exp_group, os, total_users, max_age, min_age (обратите внимание на название последних трех колонок!), используя запятую в качестве сепаратора, и отправьте файл в LMS.

Для сохранения в CSV в Redash внизу есть кнопка "три точки", где выпадает пункт "Download as CSV file".

In [35]:
df6 = pd.read_sql(
    """
SELECT 
    exp_group,
    os,
    COUNT(id) as "total_users", 
    MAX(age) as "max_age",
    MIN(age) as "min_age"
FROM "user"
GROUP BY exp_group, os
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df6.to_csv('06_DB_task6_pd.csv', index=False)
df6[:50]

,exp_group,os,total_users,max_age,min_age
0,0,Android,21234,95,14
1,0,iOS,11489,79,14
2,1,Android,21232,92,14
3,1,iOS,11406,87,14
4,2,Android,21102,78,14
5,2,iOS,11512,74,14
6,3,Android,21319,84,14
7,3,iOS,11449,85,14
8,4,Android,21085,84,14
9,4,iOS,11377,78,14


In [36]:
!cat 06_DB_task6_pd.csv

exp_group,os,total_users,max_age,min_age
0,Android,21234,95,14
0,iOS,11489,79,14
1,Android,21232,92,14
1,iOS,11406,87,14
2,Android,21102,78,14
2,iOS,11512,74,14
3,Android,21319,84,14
3,iOS,11449,85,14
4,Android,21085,84,14
4,iOS,11377,78,14


## 7

> Группировка (3/6)

- В каждой категории постов определите длину самого длинного текста, используя таблицу post. 
- В ответ отправьте тему, у которой длина самого большого поста равна по порядку 25 000.




In [40]:
df7 = pd.read_sql(
    """
SELECT
    topic,
    MAX(LENGTH(text)) as "max_lenght_text"
FROM "post"
GROUP BY topic
ORDER BY max_lenght_text DESC
LIMIT 1
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df7.head()

,topic,max_lenght_text
0,politics,25392


## 8

> Группировка (4/6)

- Найдите страну с наименьшим числом пользователей среди стран, 
- у которых общее число пользователей больше 1000. 
- Введите ее название на английском.

In [41]:
df8 = pd.read_sql(
    """
SELECT
    country,
    COUNT(id) as "count_users"
FROM "user"
GROUP BY country
HAVING COUNT(id) > 1000
ORDER BY count_users
LIMIT 1
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df8.head()

,country,count_users
0,Azerbaijan,1542


## 9

> Группировка (5/6)

Что выведет следующий запрос при его применении на таблице user?
```sql
SELECT country, exp_group, COUNT(id)
FROM "user"
GROUP BY country, exp_group;
```

`Количество пользователей внутри каждой группы, доступной в стране, для каждой страны`


In [43]:
df9 = pd.read_sql(
    """
SELECT country, exp_group, COUNT(id)
FROM "user"
GROUP BY country, exp_group
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df9.head(10)

,country,exp_group,count
0,Azerbaijan,0,341
1,Azerbaijan,1,271
2,Azerbaijan,2,327
3,Azerbaijan,3,308
4,Azerbaijan,4,295
5,Belarus,0,671
6,Belarus,1,658
7,Belarus,2,644
8,Belarus,3,641
9,Belarus,4,679


## 10

> Группировка (6/6)

- Отберите для пользователей из Москвы экспериментальные группы, 
- в которых средний возраст больше 27.2. 
- Напишите через пробел число пользователей в этих группах.



In [44]:
df10 = pd.read_sql(
    """
SELECT  
    exp_group,
    COUNT(id)
FROM "user"
WHERE city = 'Moscow'
GROUP BY exp_group
HAVING AVG(age) > 27.2
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df10.head(10)

,exp_group,count
0,1,4414
1,3,4378


In [59]:
df10['count'].to_list()

[4414, 4378]

## 11
> Сортировка (1/2)

- Сейчас мы будем работать с другой таблицей под названием post. 
- Найдите в таблице post 3 темы с наибольшим количеством постов. 
- Напишите через пробел названия этих тем.

In [60]:
df11 = pd.read_sql(
    """
SELECT 
    topic,
    COUNT(text) as "count_posts"
FROM "post"
GROUP BY topic
ORDER BY count_posts DESC
LIMIT 3
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df11.head()

,topic,count_posts
0,movie,3000
1,covid,1799
2,sport,510


In [78]:
str(df11['topic'].to_list()).replace("'", '').replace(', ',' ')

'[movie covid sport]'

## 12
> Сортировка (2/2)

- Отберите пользователей из Воронежа (Voronezh) 
- и отсортируйте их по возрасту в убывающем порядке. 
- Для равных возрастов отсортируйте по возрастанию экспериментальной группы.

- Напишите ID второго в порядке выдачи пользователя.

In [80]:
df12 = pd.read_sql(
    """
SELECT 
    id, age, exp_group
FROM "user"
WHERE city = 'Voronezh'
ORDER BY age DESC, exp_group ASC
LIMIT 2
    """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df12.head()

,id,age,exp_group
0,89990,71,0
1,2441,71,3


In [82]:
df12.loc[1][0]

2441